In [2]:
import pandas as pd
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

In [8]:
# constants from lab1
cwd = os.getcwd()

train_data_path = cwd + '/prop_data/train.csv'
test_data_path = cwd + '/prop_data/test.csv'
evaluation_data_path = cwd + '/prop_data/evaluation.csv'

In [9]:
# import data from lab1
df_train  = pd.read_csv(train_data_path)
df_test  = pd.read_csv(test_data_path)
df_evaluation  = pd.read_csv(evaluation_data_path)

In [10]:
print(df_train)

      score                                               text
0         0  overgeneralized, not helpful to anyone serious...
1         1                           Great sound and service.
2         1  love this book!!!: this book is a fast read ab...
3         1  A hugely enjoyable screen version of Rona Jaff...
4         0  What an uninteresting hodge-podge. It could ha...
5         1  @USAirways customer service at its best! Rache...
6         0  @VirginAmerica Is it normal to receive no repl...
7         0  Imagine the worst skits from Saturday Night Li...
8         0  This is one of the worst films ever. I like ch...
9         1  @JetBlue flight attendant Wendi on Flt 127 on ...
10        0  This film stars, among others, "SlapChop" Vinc...
11        0  @united This isn't a one time thing either! It...
12        1  It seems a lot of IMDB comments on this film a...
13        1  I always wondered what happened with that magi...
14        1  eye opening: when i think of poetry i put 

In [17]:
 #from lab1
import re
import string

# data preprocessing, same as lab 1
def preprocess_text(text):
    # replacing url-s with the word 'url'
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL', text)
    # replacing usernames-s with the word 'user'
    text = re.sub('@[^\s]+','USER', text)
    # converting text to lowercase
    text = text.lower()
    # remove multiple spaces
    text = re.sub(' +',' ', text)
    # remove punctuation marks
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    # replacing numbers with the word 'number'
    text = re.sub(r'\w*\d+\w*', 'number', text)
    return text.strip()

tr_data = [preprocess_text(t) for t in df_train.text]
test_data = [preprocess_text(t) for t in df_test.text]
evaluation_data = [preprocess_text(t) for t in df_evaluation.text]

#Since the preprocessing of the texts create a list of texts, we create dataframes again to fit to the rest of the code


tr_data = pd.DataFrame(tr_data, columns =['text'])
test_data = pd.DataFrame(test_data, columns =['text'])
evaluation_data = pd.DataFrame(evaluation_data, columns =['text'])


#After tr_data, test_data and evaluation_data got turned into dataframes, we take the columns of these 
#and add to the original dataframes, meaning they also contain their scores
df_train['text'] = tr_data['text'].values
df_test['text'] = test_data['text'].values
df_evaluation['text'] = evaluation_data['text'].values

print(df_train)

      score                                               text
0         0  overgeneralized not helpful to anyone seriousl...
1         1                            great sound and service
2         1  love this book this book is a fast read about ...
3         1  a hugely enjoyable screen version of rona jaff...
4         0  what an uninteresting hodge podge it could hav...
5         1  user customer service at its best rachel s too...
6         0  user is it normal to receive no reply from cen...
7         0  imagine the worst skits from saturday night li...
8         0  this is one of the worst films ever i like che...
9         1  user flight attendant wendi on flt number on n...
10        0  this film stars among others slapchop vince of...
11        0  user this isn t a one time thing either it s a...
12        1  it seems a lot of imdb comments on this film a...
13        1  i always wondered what happened with that magi...
14        1  eye opening when i think of poetry i put r

In [18]:
#Tutorial from https://medium.com/@vasista/sentiment-analysis-using-svm-338d418e3ff1, with their standard values

from sklearn.feature_extraction.text import TfidfVectorizer
# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

train_vectors = vectorizer.fit_transform(df_train["text"])
test_vectors = vectorizer.transform(df_test["text"])
evaluation_vectors = vectorizer.transform(df_evaluation["text"])

In [19]:
#Tutorial from https://medium.com/@vasista/sentiment-analysis-using-svm-338d418e3ff1 also

import time
from sklearn import svm
from sklearn.metrics import classification_report
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, df_train['score'])
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
prediction_linear2 = classifier_linear.predict(evaluation_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1
# results


print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(df_test['score'], prediction_linear, output_dict=True)
report2 = classification_report(df_evaluation['score'], prediction_linear2, output_dict=True)
print('positive: ', report['1'])
print('negative: ', report['0'])

print('positive: ', report2['1'])
print('negative: ', report2['0'])


Training time: 12.938633s; Prediction time: 16.045002s
positive:  {'precision': 0.856909239574816, 'recall': 0.8397435897435898, 'f1-score': 0.848239579117766, 'support': 1248}
negative:  {'precision': 0.8433829287392326, 'recall': 0.860223642172524, 'f1-score': 0.8517200474495848, 'support': 1252}
positive:  {'precision': 0.858540562576437, 'recall': 0.8363780778395552, 'f1-score': 0.8473144236572118, 'support': 2518}
negative:  {'precision': 0.8382410679230468, 'recall': 0.8601933924254633, 'f1-score': 0.8490753628952078, 'support': 2482}
